In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [26]:
#Load Data
df_final_demo = pd.read_csv("https://drive.google.com/uc?id=1n-do3beXJIXcVdcUp8gAhIb4jbSjomWK")
df_final_exp_clients = pd.read_csv("https://drive.google.com/uc?id=19mSpzew88dgaSQKvpbS1Dt0L7h-WKk50")
df_web_data_1 = pd.read_csv("https://drive.google.com/uc?id=1sTNwBd9Gr2_v3eNuAH-YbZ7Jh6YrKTPD")
df_web_data_2 = pd.read_csv("https://drive.google.com/uc?id=1dUbosgeNvDPAHe-i761aF-YH20raALek")

#Concat Web Data
df_web = pd.concat([df_web_data_1, df_web_data_2], ignore_index = True)

In [27]:
#Convert Timestamps (It converts the values in the column date_time into real datetime objects that Python can understand and work with)
df_web['date_time'] = pd.to_datetime(df_web['date_time'], errors = 'coerce')

In [28]:
# Sort for step duration calculation (It sorts the dataset so that, for each client and each visit, the rows (steps) are ordered in chronological order
# For example, .diff() calculates the time difference between the current step and the previous step, but it only works correctly if the rows are already in order)
df_web = df_web.sort_values(by = ['client_id', 'visit_id', 'date_time'])

In [29]:
# Step duration (seconds) per step per visit
df_web['time_spent_sec'] = df_web.groupby(['client_id', 'visit_id'])['date_time'].diff().dt.total_seconds()
df_web['time_spent_sec'] = df_web['time_spent_sec'].fillna(0)

In [30]:
#Merging DF_Web with the other 2 dataframes
df_master = pd.merge(df_web, df_final_exp_clients, on='client_id', how='left')
df_master = pd.merge(df_master, df_final_demo, on='client_id', how='left')
df_master

,client_id,visitor_id,visit_id,process_step,date_time,time_spent_sec,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,169,201385055_71273495308,749567106_99161211863_557568,start,2017-04-12 20:19:36,0.0,NaN,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0
1,169,201385055_71273495308,749567106_99161211863_557568,step_1,2017-04-12 20:19:45,9.0,NaN,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0
2,169,201385055_71273495308,749567106_99161211863_557568,step_2,2017-04-12 20:20:31,46.0,NaN,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0
3,169,201385055_71273495308,749567106_99161211863_557568,step_3,2017-04-12 20:22:05,94.0,NaN,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0
4,169,201385055_71273495308,749567106_99161211863_557568,confirm,2017-04-12 20:23:09,64.0,NaN,21.0,262.0,47.5,M,2.0,501570.72,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755400,9999875,738878760_1556639849,931268933_219402947_599432,step_1,2017-06-01 22:40:08,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
755401,9999875,738878760_1556639849,931268933_219402947_599432,step_1,2017-06-01 22:41:28,80.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
755402,9999875,738878760_1556639849,931268933_219402947_599432,step_2,2017-06-01 22:41:47,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
755403,9999875,738878760_1556639849,931268933_219402947_599432,step_3,2017-06-01 22:44:58,191.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Remove duplicates
df_master = df_master.drop_duplicates(subset=['client_id','visit_id','process_step','date_time'])

df_master
# Drop rows missing Variation
df_master = df_master.dropna(subset=['Variation'])

In [32]:
# Session minutes per visit
df_master['session_minutes'] = (
    df_master.groupby(['client_id','visit_id'])['date_time'].transform('max') -
    df_master.groupby(['client_id','visit_id'])['date_time'].transform('min')
).dt.total_seconds() / 60

# Remove Outliers
df_master = df_master[df_master['session_minutes'] < df_master['session_minutes'].quantile(0.99)]

In [33]:
# Step order mapping
step_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']
step_rank = {step:i for i, step in enumerate(step_order, start=1)}
df_master['step_num'] = df_master['process_step'].map(step_rank)

# Completion & error flags
max_step_num = df_master['step_num'].max()
df_master['completed'] = (df_master['step_num'] == max_step_num).astype(int)
df_master['error'] = (df_master['step_num'] < max_step_num).astype(int)

KPIs

In [42]:
# 1. Completion rate
completion_rate = df_master.groupby('Variation')['completed'].mean() * 100

# 2.1 Average session time
avg_session_time = df_master.groupby('Variation')['session_minutes'].mean()

# 2.2 Average time spent per step (minutes)
avg_time_per_step = df_master.groupby(['Variation', 'process_step'])['time_spent_sec'].mean().reset_index()
avg_time_per_step['time_spent_min'] = avg_time_per_step['time_spent_sec'] / 60
avg_time_per_step.rename(columns={'time_spent_min': 'Avg Time per Step (min)'}, inplace=True)
avg_time_clean = avg_time_per_step[['Variation', 'process_step', 'Avg Time per Step (min)']]

# 3. Error rate
error_rate = df_master.groupby('Variation')['error'].mean() * 100

# 4. Drop-off per step (%)
step_counts = df_master.groupby(['Variation','process_step'])['client_id'].nunique().unstack('Variation').reindex(step_order)
step_percentage = (step_counts / step_counts.iloc[0]) * 100
drop_off = step_percentage.diff(-1).abs()
drop_off_long = drop_off.reset_index().melt(
    id_vars='process_step', 
    var_name='Variation', 
    value_name='Drop-Off Rate (%)'
)

KPI Summary Table

In [43]:
kpi_summary = pd.DataFrame({
    'Completion Rate (%)': completion_rate.round(2),
    'Avg Session Time (min)': avg_session_time.round(2),
    'Error Rate (%)': error_rate.round(2)
}).reset_index()
combined_steps = pd.merge(
    drop_off_long,
    avg_time_clean,
    on=['Variation', 'process_step'],
    how='left'
)
kpi_combined = pd.merge(
    combined_steps,
    kpi_summary,
    on='Variation',
    how='left'
)
kpi_combined = kpi_combined.sort_values(by=['Variation', 'process_step']).reset_index(drop=True)


kpi_combined

,process_step,Variation,Drop-Off Rate (%),Avg Time per Step (min),Completion Rate (%),Avg Session Time (min),Error Rate (%)
0,confirm,Control,NaN,2.011700,12.35,6.75,87.65
1,start,Control,13.930391,0.813931,12.35,6.75,87.65
2,step_1,Control,6.434351,0.692948,12.35,6.75,87.65
3,step_2,Control,5.244231,0.638790,12.35,6.75,87.65
4,step_3,Control,8.476390,1.542014,12.35,6.75,87.65
5,confirm,Test,NaN,1.693113,14.46,7.14,85.54
6,start,Test,9.111437,0.896373,14.46,7.14,85.54
7,step_1,Test,7.594746,0.595909,14.46,7.14,85.54
8,step_2,Test,5.174815,0.754077,14.46,7.14,85.54
9,step_3,Test,8.106582,1.532196,14.46,7.14,85.54


Hyphothesis

Hypothesis 1 — Completion Rate

H0:
There is no significant difference in the completion rate between the Control and Test designs.

H1:
There is a significant difference in the completion rate between the Control and Test designs.

Hypothesis 2 — Average Time Spent per Step

H0:
There is no significant difference in the average time spent per step between the Control and Test designs.

H1:
There is a significant difference in the average time spent per step between the two designs.

Hypothesis 3 — Drop-off Rates

H0:
The proportion of users who abandon the process at each step is the same for both designs.

H1:
The proportion of users who abandon the process differs between designs.

Hypothesis 4 — Error Rate

H0:
There is no difference in the error rate (percentage of incomplete processes) between the Control and Test designs.

H1:
There is a difference in error rates between the two designs.

In [36]:
from scipy.stats.contingency import association
from scipy.stats import chi2_contingency
from scipy.stats import ttest_ind


In [37]:
#Hypothesis 1 — Completion Rate
from scipy.stats import chi2_contingency

# Crosstab for the Chi-square test
crosstab_result = pd.crosstab(df_master['Variation'], df_master['completed'])

# Run Chi-square test (since both variables are categorical)
_, chi2_p_value, _, _ = chi2_contingency(crosstab_result)

# Display results
print("p-value:", chi2_p_value)

# Interpretation
#Since chi2_p_value >= 0.05 indicates that there is not a significant difference in completion rate between Control and Test.

p-value: 3.5447771754378953e-66


In [38]:
# Hypothesis 2 — Average Time Spent per Session
from scipy.stats import ttest_ind

# Select groups
df_control = df_master[df_master["Variation"] == "Control"]["session_minutes"]
df_test = df_master[df_master["Variation"] == "Test"]["session_minutes"]

# Run Two Sample T-test
_, p_value = ttest_ind(df_control, df_test)

print("p-value:", p_value) 

# Interpretation
#Since p_value >= 0.05 indicates that there is not a significant difference in average time spent per session between Control and Test.

p-value: 2.409316898539708e-43


In [39]:
#3 Hypothesis testing
#3.1
control=step_counts.loc['step_1','Control']
test= step_counts.loc['step_1', 'Test']

control_start = step_counts.loc['start', 'Control']
test_start = step_counts.loc['start', 'Test']

control_drop=control_start-control
test_drop=test_start-test
table = [[control, control_drop],[test, test_drop]]
chi2,p,_,_=chi2_contingency(table)
print(f"Chi2 {chi2}, p value {p}")
#3.2
control2 = step_counts.loc['step_2', 'Control']
test2= step_counts.loc['step_2', 'Test']

control_drop2=control-control2
test_drop2=test-test2

table2 = [[control2, control_drop2],[test2, test_drop2]]
chi2,p,_,_=chi2_contingency(table2)
print(f"Chi2 {chi2}, p value {p}")
#3.3
control3 = step_counts.loc['step_3', 'Control']
test3= step_counts.loc['step_3', 'Test']

control_drop3=abs (control2-control3)
test_drop3=abs(test2-test3)

table3 = [[control3, control_drop3],[test3, test_drop3]]
chi2,p,_,_=chi2_contingency(table3)
print(f"Chi2 {chi2}, p value {p}")

#3.4
control_confirm = step_counts.loc['confirm', 'Control']
test_confirm = step_counts.loc['confirm', 'Test']

control_drop_confirm=abs (control3-control_confirm)
test_drop_confirm=abs(test3-test_confirm)

table4 = [[control_confirm, control_drop_confirm],[test_confirm, test_drop_confirm]]
chi2,p,_,_=chi2_contingency(table4)
print(f"Chi2 {chi2}, p value {p}")

#Cramer Vs for the significantly ones
v1 = association(table, method="cramer")
v2 = association(table2, method="cramer")
v4 = association(table4, method="cramer")

print(f"The Camers V for start to 1 is {v1}")
print(f"The Camers V for the step 1 to 2 is {v2}")
print(f"The Camers V for the step 3 to confirm is {v4}")

# Interpretation

#Statistical significance was found in three out of four transitions (start→1, 1→2, 3→confirm).
#However, Cramer’s V values are below 0.1, indicating that the practical effect is small — the new design does influence user flow, but only slightly.
#The largest difference occurs at the start→step 1 transition, meaning that the new interface may encourage more users to begin the process.

Chi2 286.072225688146, p value 3.567817307648024e-64
Chi2 11.49127967330664, p value 0.000699234780049255
Chi2 2.286143723358041, p value 0.13053388581355066
Chi2 10.019504649546306, p value 0.001548911198149801
The Camers V for start to 1 is 0.07575646653404626
The Camers V for the step 1 to 2 is 0.016197838963967637
The Camers V for the step 3 to confirm is 0.01629408157533467


In [40]:
#4 Hypothesis testing
crosstab=pd.crosstab(df_master['Variation'],df_master['completed'])
crosstab
#Not completed= 0 
#completed =1
chi2, p, dof, expected = chi2_contingency(crosstab)
print(f"Chi2 Statistic: {chi2:.3f}")
print(f"Degrees of Freedom: {dof}")
print(f"P-value: {p:.5f}")

association(crosstab, method="cramer")

#Interpretations

#The Chi2 test demonstrates a statistically significant association between design version and completion rate (chi2 = 295.26, p < 0.001).
#Therefore, the null hypothesis is rejected: the completion rate differs between the Control and Test groups.
#Although there is a statically significant value, the association strenght is very weak

Chi2 Statistic: 295.264
Degrees of Freedom: 1
P-value: 0.00000


0.030671755429582573

In [41]:
# MAYBE DELETE??????????????????????????



#Variation fillna with mode
mode_value=df_master["Variation"].mode(dropna=True)[0]
df_master["Variation"].fillna(mode_value, inplace=True)

#clnt_tenure_yr  & clnt_tenure_mnth
df_master[["clnt_tenure_yr", "clnt_tenure_mnth"]].isna().sum()
only_one_missing = ((df_master["clnt_tenure_yr"].isna()) ^ (df_master["clnt_tenure_mnth"].isna())).sum()
print(f" one missing: {only_one_missing}")

cols= ["clnt_tenure_yr", "clnt_tenure_mnth"]
for col in cols:
    clean_data=df_master[col].dropna()

    mean = clean_data.mean()
    std = clean_data.std()
    median = clean_data.median()
    mode = clean_data.mode()[0]
    n_missing=df_master[col].isna().sum()

    if n_missing > 0:
        #filll values without changing the stats of the column, it fill it with random values and 
        random_fill = np.random.normal(loc=mean, scale=std, size=n_missing)
        #limit the values to a range 
        random_fill = np.clip(random_fill, clean_data.min(), clean_data.max())
        df_master.loc[df_master[col].isna(), col] = random_fill

#clnt_age  
col_age=["clnt_age"]
for col in col_age:
    clean_age=df_master[col].dropna()  

    mean = clean_age.mean()
    std = clean_age.std()
    median = clean_age.median()
    mode = clean_age.mode()[0]
    n_missing=df_master[col].isna().sum()

    if n_missing > 0:
        random_fill = np.random.normal(loc=mean, scale=std, size=n_missing)
        df_master.loc[df_master[col].isna(), col] = random_fill
print(df_master[col_age].isna().sum())

 one missing: 0
clnt_age    0
dtype: int64


C:\Users\claud\AppData\Local\Temp\ipykernel_20444\2289080212.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_master["Variation"].fillna(mode_value, inplace=True)
